In [1]:
# import libraries
import pandas as pd
import numpy as np
import math as mt
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

In [11]:
try:
    df = pd.read_csv(data_file)
except Exception as e:
    print(e) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


In [2]:
# Utilities
def to_dollars(value):
    return '${:,.2f}'.format(value)

def d_from_l(center, target, kms=True):
    
    # Converted from https://andrew.hedges.name/experiments/haversine/    
    if kms:
        R = 6373
    else:
        R = 3961
    
    dlon = target[1] - center[1] 
    dlat = target[0] - center[0] 
    a = (mt.sin(dlat/2))^2 + mt.cos(lat1) * mt.cos(lat2) * (mt.sin(dlon/2))^2 
    c = 2 * mt.atan2( mt.sqrt(a), mt.sqrt(1-a) ) 
    d = R * c
    return d

def get_data(data_file):
    try:
        data = pd.read_csv(data_file)
    except Exception as e:
        print(e) 
    return data

def get_clean_data(data):
    columns = ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
    'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
    'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
    'lat', 'long', 'sqft_living15', 'sqft_lot15']


# flr_dummies = pd.get_dummies(data["floors"], prefix="flr")
#     data = pd.concat([data.loc[:,'price':'floors'], flr_dummies], axis=1)
#     data.drop('floors', axis=1, inplace = True)
#     data.head()
    
    data.date = pd.to_datetime(data.date, format='%m/%d/%Y')
#     return pd.concat([data.loc[:,'price':'floors'], data.date], axis=1) 
#     return data.loc[:,'price':'floors']
    
    return data.drop(['id', 'date', 'sqft_basement'], axis=1)
#     tried = ['sqft_lot15', 'sqft_living15', 'long','lat', 'zipcode', 'yr_built']

# def transform_data(dat):
#     import pandas as pd
#     import numpy as np
#     data = pd.read_csv("auto-mpg.csv") 
#     data['horsepower'].astype(str).astype(int)

#     acc = data["acceleration"]
#     logdisp = np.log(data["displacement"])
#     loghorse = np.log(data["horsepower"])
#     logweight= np.log(data["weight"])

#     scaled_acc = (acc-min(acc))/(max(acc)-min(acc))	
#     scaled_disp = (logdisp-np.mean(logdisp))/np.sqrt(np.var(logdisp))
#     scaled_horse = (loghorse-np.mean(loghorse))/(max(loghorse)-min(loghorse))
#     scaled_weight= (logweight-np.mean(logweight))/np.sqrt(np.var(logweight))

#     data_fin = pd.DataFrame([])
#     data_fin["acc"]= scaled_acc
#     data_fin["disp"]= scaled_disp
#     data_fin["horse"] = scaled_horse
#     data_fin["weight"] = scaled_weight
#     cyl_dummies = pd.get_dummies(data["cylinders"], prefix="cyl")
#     yr_dummies = pd.get_dummies(data["model year"], prefix="yr")
#     orig_dummies = pd.get_dummies(data["origin"], prefix="orig")
#     mpg = data["mpg"]
#     data_fin = pd.concat([mpg, data_fin, cyl_dummies, yr_dummies, orig_dummies], axis=1)


def split_data(data, sample):
    cutoff = int(len(data)*sample)
    return (data[:cutoff],data[cutoff:])

def split_target(data, col_name):
    return data[col_name], data.drop(col_name, axis=1)

In [8]:
data_file = 'kc_house_data.csv'
df = get_data(data_file)
print(df.columns)
df.sqft_basement.describe()

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')


count     21597
unique      304
top         0.0
freq      12826
Name: sqft_basement, dtype: object

In [ ]:
# Define your project variables
data_file = 'kc_house_data.csv'
sample_percent = 0.8

#Scrape, clean and split the data into two sets
(df, test_df) = split_data(get_clean_data(get_data(data_file)), sample_percent)

#Split the columns into target and features for OLS
y, X = split_target(df, 'price')

In [ ]:
for column in df:
    df[column].plot.hist(density=True, label = column+' histogram')
    df[column].plot.kde(label=column+' kde')
    plt.legend()
    plt.show()

In [ ]:
df.bas

In [ ]:
X_int = sm.add_constant(X)
model = sm.OLS(y,X_int).fit()
model.summary()